# EXTRACT 

In [ ]:
import awswrangler as awr
import logging
import pandas as pd
import os

logging.basicConfig(
    level=logging.INFO,  # Exibe mensagens a partir de INFO
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        logging.StreamHandler()  # Garante logs no console
    ]
)

logging.info('\n ----------------------------------------------------------------------------------')
logging.info('\n Executando Rotina: Movimentação de Placas')

class Extract:

    def __init__(self):
        self.path = r"C:\Users\raphael.almeida\Documents\Processos\placas_movimentacoes"

    def extract_all_ativacoes(self):

        try:
            dir_query = os.path.join(self.path, 'sql', 'all_boards_ATIVOS.sql')
            with open(dir_query, 'r') as file:
                query = file.read()
            df_ativacoes = awr.athena.read_sql_query(query, database='silver')
            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info('\n Consulta de ativações extraída com sucesso!')
            return df_ativacoes
        except Exception as e:
            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info(f'\n Falha ao extrair a consulta de ativações: {e}')
            return None

    def extract_all_cancelamentos(self):

        try:
            dir_query = os.path.join(self.path, 'sql', 'all_boards_CANCELADOS.sql')
            with open(dir_query, 'r') as file:
                query = file.read()
            df_cancelamentos = awr.athena.read_sql_query(query, database='silver')
            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info('\n Consulta de cancelamentos extraída com sucesso!')
            return df_cancelamentos
        except Exception as e:
            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info(f'\n Falha ao extrair a consulta de cancelamentos: {e}')
            return None

    def extract_conf_boards(self):

        try:

            dir_query = os.path.join(self.path,'sql', 'listagem_mestra.sql')

            with open(dir_query, 'r') as file:
                query = file.read()

            df_conferencia = awr.athena.read_sql_query(query, database='silver')
        
            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info('\n Consulta de dados históricos realizada com sucesso!')

            return df_conferencia

        except Exception as e:

            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info(f'\n Falha ao extrair consulta de dados históricos: {e}')



extract = Extract()
df_ativacoes = extract.extract_all_ativacoes()
df_cancelamentos = extract.extract_all_cancelamentos()
df_conf = extract.extract_conf_boards()



# TRANSFORM

board_status_treatment


In [ ]:
# IMPORTANDO MÓDULOS E PACOTES
import pandas as pd
import numpy as np
import datetime as dt
import logging
import numpy as np

class Transform:
        
    def __init__(self) -> None:
        pass

    # CRIANDO FUNÇÃO QUE IRÁ APLICAR O TRATAMENTO DE STATUS DAS PLACAS ATIVADAS
    def board_status_treatment(df, df_ontem, df_conf, status_filter_list):

        try:
            status_filter_list = ['CANCELADO', 'CANCELADA', 'FINALIZADO', 'FINALIZADA', 'NAO RENOVADO']

            if not df.empty:
                logging.info('\n ----------------------------------------------------------------------------------')
                logging.info(df.shape)
                row_count = 0
                for idx, row in df.iterrows():
                    row_count += 1
                    df_comp_ontem = df_ontem[df_ontem['chassi'] == row['chassi']]
                    df_comp_conf = df_conf[(df_conf['chassi'] == row['chassi']) & (df_conf['beneficio'] == row['beneficio'])].sort_values(by='data_ativacao', ascending=False)

                    if not df_comp_ontem.empty:                                        
                                if df_comp_ontem['empresa']!=row['empresa']:
                                    df.at[idx, 'status_beneficio'] = 'MIGRAÇÃO'
                                    df.at[idx, 'migration_from'] = 'NULL'
                                else:
                                    df.at[idx, 'status_beneficio'] = 'ATIVO'
                                    df.at[idx, 'migration_from'] = 'NULL'                     
                    else:
                        hist_datas_ativacao = sorted(df_comp_conf['data_ativacao_beneficio'].dropna().drop_duplicates().unique())                    
                        if len(hist_datas_ativacao) > 1:
                            penultimo_registro_data = hist_datas_ativacao[-2]
                            verification_penultima_row = df_comp_conf.loc[df_comp_conf['data_ativacao_beneficio'] == penultimo_registro_data]
                            
                            if verification_penultima_row['status_beneficio'].values[0] not in status_filter_list:
                                if verification_penultima_row['empresa'].values[0] != row['empresa']:
                                    df.at[idx, 'status_beneficio'] = 'MIGRAÇÃO' #EM ALGUM MOMENTO ELA ESTAVA COMO ATIVA NA OUTRA EMPRESA, NÃO FOI CANCELADA, MAS TAMBÉM NÃO ESTAVA ATIVA
                                    df.at[idx, 'migration_from'] = verification_penultima_row['empresa'].values[0]
                                else:
                                    df.at[idx, 'status_beneficio'] = 'RENOVAÇÃO' #NÃO TINHA STATUS DE CANCELAMENTO NA EMPRESA, MAS TAMBÉM NÃO ESTAVA ATIVA
                                    df.at[idx, 'migration_from'] = 'NULL'
                            else:
                                    df.at[idx, 'status_beneficio'] = 'REATIVAÇÃO'
                                    df.at[idx, 'migration_from'] = 'NULL'
                                    
                        else:
                            df.at[idx, 'status_beneficio'] = 'NOVO'
                            df.at[idx, 'migration_from'] = 'NULL'

                logging.info('\n ----------------------------------------------------------------------------------')
                logging.info(f'Total de linhas processadas: {row_count}')

            else:
                logging.info('\n ----------------------------------------------------------------------------------')
                logging.info('Nenhum registro de ativações para tratamento de dados. Dataframe vazio!')


        except Exception as e:

            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info(f'Falha no tratamento de status das placas ativadas. Revise o código: {e}')

        return df

transform 

In [7]:
# IMPORTANDO MÓDULOS E PACOTES
import pandas as pd
import numpy as np
import datetime as dt
import logging


class Transform:
        
    def __init__(self) -> None:
        pass

    def transforming_files(self):

#DEFININDO DATAFRAMES VAZIOS 
        try:

            df_final_ativacoes = pd.DataFrame()
            df_final_cancelamentos = pd.DataFrame()

        except Exception as e:

            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info('Falha ao definir dataframes.')

#TRANSFORMANDO DF_ATIV E SEGMENTANDO POR EMPRESA
        try:
            df_ativ_all_boards=df_ativacoes
            df_final_cancelamentos=df_cancelamentos
            
            #extract = Extract()
            #df_final_cancelamentos = extract.extract_all_cancelamentos()
            #df_ativ_all_boards = extract.extract_all_ativacoes()

            df_ativ_all_boards['data_ativacao_beneficio'] = pd.to_datetime(df_ativ_all_boards['data_ativacao_beneficio']).dt.date
                        
            df_ativ_all_boards['beneficio'] = df_ativ_all_boards['beneficio'].replace('REPARAÇÃO OU REPOSIÇÃO DO VEÍCULO', 'CASCO (VEÍCULO)').replace('REPARAÇÃO OU REPOSIÇÃO DO (SEMI)REBOQUE', 'CASCO (R/SR)').replace('REPARAÇÃO OU REPOSIÇÃO DO COMPLEMENTO', 'CASCO (COMPLEMENTO)')
            
            df_ativ_viavante = df_ativ_all_boards[df_ativ_all_boards['empresa'] == 'Viavante']
            df_ativ_stcoop = df_ativ_all_boards[df_ativ_all_boards['empresa'] == 'Stcoop']
            df_ativ_segtruck = df_ativ_all_boards[df_ativ_all_boards['empresa'] == 'Segtruck']
            df_ativ_tag = df_ativ_all_boards[df_ativ_all_boards['empresa'] == 'Tag']

            df_final_cancelamentos = df_final_cancelamentos

        except Exception as e:

            logging.info('\n ----------------------------------------------------------------------------------')  
            logging.info(f'Falha ao realizar a segmentação dos dataframes: {e}')

# SELECIONANDO APENAS AS ATIVAÇÕES CORRESPONDENTES AOS BENEFICIOS 'CASCO' / 'TERCEIRO' POR UM REGEX PADRÃO
        try:
            ids_beneficios_segtruck = [2, 3, 4, 7, 24, 25, 26, 29]
            ids_beneficios_stcoop = [24, 25, 26, 29]
            ids_beneficios_viavante = [40, 41, 42, 45]
            ids_beneficios_tag = [2, 3, 4, 7, 24, 25, 26, 29, 34, 35, 36, 37, 38, 39]

            df_ativ_viavante = df_ativ_viavante.loc[df_ativ_viavante['id_beneficio'].isin(ids_beneficios_viavante)]
            df_ativ_stcoop = df_ativ_stcoop.loc[df_ativ_stcoop['id_beneficio'].isin(ids_beneficios_stcoop)]
            df_ativ_segtruck = df_ativ_segtruck.loc[df_ativ_segtruck['id_beneficio'].isin(ids_beneficios_segtruck)]
            df_ativ_tag = df_ativ_tag.loc[df_ativ_tag['id_beneficio'].isin(ids_beneficios_tag)]

        except Exception as e:

            logging.info('\n ----------------------------------------------------------------------------------')  
            logging.info(f'Falha ao padronizar nomenclaturas referente aos beneficios pré-estabelecidos: {e}')

# CONCATENANDO E CRIANDO COLUNA DE MIGRAÇÃO (MIGRATION_FROM) 
        try:

            df_final_ativacoes = pd.concat([df_ativ_viavante, df_ativ_stcoop, df_ativ_segtruck, df_ativ_tag])

            if not df_final_ativacoes.empty:
                df_final_ativacoes['migration_from'] = np.nan

        except Exception as e:
            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info(f'Falha na criação da coluna de migração e concatenação de dataframes: {e}')

    # DEFININDO COLUNAS QUE SERÃO UTILIZADAS NO DATAFRAME FINAL
        try:
           
            df_final_ativacoes = df_final_ativacoes[[
                'placa', 'chassi', 'id_placa', 'id_veiculo', 'id_carroceria', 'matricula', 'conjunto', 'unidade', 'consultor', 'status_beneficio', 
                'cliente', 'data_registro', 'data_ativacao_beneficio', 'suporte', 'data_filtro', 'empresa', 'migration_from'
            ]]

            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info(f'Processo de seleção de colunas realizado com sucesso!')

        except Exception as e:
            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info(f'Falha ao definir colunas: {e}')

    # RETIRANDO DUPLICATAS
        try:
            df_final_ativacoes = df_final_ativacoes.drop_duplicates(subset=['chassi'])

            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info(f'Duplicatas retiradas com sucesso.')

        except Exception as e:
            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info(f'Falha ao retirar duplicatas. Revise o código: {e}')

# TRATANDO DADOS NULOS NOS DATAFRAMES
        try:
            df_final_ativacoes['placa'] = df_final_ativacoes['placa'].fillna('SEM-PLACA')
            df_final_ativacoes['chassi'] = df_final_ativacoes['chassi'].fillna('NULL')
            df_final_ativacoes['id_placa'] = df_final_ativacoes['id_placa'].fillna(0)
            df_final_ativacoes['id_veiculo'] = df_final_ativacoes['id_veiculo'].fillna(0)
            df_final_ativacoes['id_carroceria'] = df_final_ativacoes['id_carroceria'].fillna(0)
            df_final_ativacoes['matricula'] = df_final_ativacoes['matricula'].fillna(0)
            df_final_ativacoes['conjunto'] = df_final_ativacoes['conjunto'].fillna(0)
            df_final_ativacoes['unidade'] = df_final_ativacoes['unidade'].fillna('NULL')
            df_final_ativacoes['consultor'] = df_final_ativacoes['consultor'].fillna('NULL')
            df_final_ativacoes['status_beneficio'] = df_final_ativacoes['status_beneficio'].fillna('NULL')
            df_final_ativacoes['cliente'] = df_final_ativacoes['cliente'].fillna('NULL')
            df_final_ativacoes['data_registro'] = df_final_ativacoes['data_registro'].fillna(pd.Timestamp('1900-01-01'))
            df_final_ativacoes['data_ativacao_beneficio'] = df_final_ativacoes['data_ativacao_beneficio'].fillna(pd.Timestamp('1900-01-01'))
            df_final_ativacoes['suporte'] = df_final_ativacoes['suporte'].fillna('NULL')
            df_final_ativacoes['data_filtro'] = df_final_ativacoes['data_filtro'].fillna(pd.Timestamp('1900-01-01'))
            df_final_ativacoes['empresa'] = df_final_ativacoes['empresa'].fillna('NULL')
            df_final_ativacoes['migration_from'] = df_final_ativacoes['migration_from'].fillna('NULL')

            df_final_cancelamentos['placa'] = df_final_cancelamentos['placa'].fillna('SEM-PLACA')
            df_final_cancelamentos['chassi'] = df_final_cancelamentos['chassi'].fillna('NULL')
            df_final_cancelamentos['id_placa'] = df_final_cancelamentos['id_placa'].fillna(0)
            df_final_cancelamentos['id_veiculo'] = df_final_cancelamentos['id_veiculo'].fillna(0)
            df_final_cancelamentos['id_carroceria'] = df_final_cancelamentos['id_carroceria'].fillna(0)
            df_final_cancelamentos['matricula'] = df_final_cancelamentos['matricula'].fillna(0)
            df_final_cancelamentos['conjunto'] = df_final_cancelamentos['conjunto'].fillna(0)
            df_final_cancelamentos['unidade'] = df_final_cancelamentos['unidade'].fillna('NULL')
            df_final_cancelamentos['status'] = df_final_cancelamentos['status'].fillna('NULL')
            df_final_cancelamentos['cliente'] = df_final_cancelamentos['cliente'].fillna('NULL')
            df_final_cancelamentos['data'] = df_final_cancelamentos['data'].fillna(pd.Timestamp('1900-01-01'))
            df_final_cancelamentos['data_cancelamento'] = df_final_cancelamentos['data_cancelamento'].fillna(pd.Timestamp('1900-01-01'))
            df_final_cancelamentos['usuario_cancelamento'] = df_final_cancelamentos['usuario_cancelamento'].fillna('NULL')
            df_final_cancelamentos['data_filtro'] = df_final_cancelamentos['data_filtro'].fillna(pd.Timestamp('1900-01-01'))
            df_final_cancelamentos['empresa'] = df_final_cancelamentos['empresa'].fillna('NULL')
            df_final_cancelamentos['migracao'] = df_final_cancelamentos['migracao'].fillna('NULL')
            df_final_cancelamentos['renegociacao'] = df_final_cancelamentos['renegociacao'].fillna('NULL')
            df_final_cancelamentos['data_substituicao'] = df_final_cancelamentos['data_substituicao'].fillna(pd.Timestamp('1900-01-01'))

            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info('\n Processo de Transformacao de Dados concluido com sucesso!')

        except Exception as e:
            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info(f'Falha ao realizar tratamento de dados: {e}')
     
            return df_final_ativacoes, df_final_cancelamentos
    
transform = Transform()
df_final_ativacoes, df_final_cancelamentos = transform.transforming_files()

        


board_status_treatment2

declaring variables

In [55]:
# IMPORTANDO MÓDULOS E PACOTES
import pandas as pd
import numpy as np
import datetime as dt
import logging
import numpy as np
import os
import awswrangler as awr

xlsx_ontem = r"C:\Users\raphael.almeida\Documents\Processos\placas_movimentacoes\placas_movimentacoes_ontem.xlsx"
xlsx = r"C:\Users\raphael.almeida\Documents\Processos\placas_movimentacoes\placas_movimentacoes.xlsx"

df_ontem = pd.read_excel(xlsx_ontem, engine='openpyxl', sheet_name= 'ATIVAÇÕES')
df = pd.read_excel(xlsx, engine='openpyxl')
status_filter_list = ['CANCELADO', 'CANCELADA', 'FINALIZADO', 'FINALIZADA', 'NAO RENOVADO']

path = r"C:\Users\raphael.almeida\Documents\Processos\placas_movimentacoes"
dir_query = os.path.join(path,'sql', 'listagem_mestra.sql')

with open(dir_query, 'r') as file:
    query = file.read()

df_conferencia = awr.athena.read_sql_query(query, database='silver')



In [56]:
print(df.shape[0])
print(df_ontem.shape[0])

32908
33250


In [58]:
df_novos = df[df['status_beneficio']=='NOVO']
df_ativos = df[df['status_beneficio']=='ATIVO']

print(df_novos.shape[0])
print(df_ativos.shape[0])



0
32908


In [53]:
# Construir df_cancelados: existiam ontem e não existem hoje
set_hoje = set(df['chassi'].dropna().unique()) if 'chassi' in df.columns else set()

if 'chassi' in df_ontem.columns:
    cancelados_mask = ~df_ontem['chassi'].isin(set_hoje)
    df_cancelados = df_ontem.loc[cancelados_mask].copy()
else:
    df_cancelados = pd.DataFrame(columns=df_ontem.columns)

In [54]:
df_cancelados = pd.DataFrame(df_cancelados)
df_cancelados.shape[0]

448

In [ ]:


xlsx_cancelation_path = r"C:\Users\raphael.almeida\Documents\Processos\placas_movimentacoes\df_cancelamentos.xlsx"
df_cancelamentos = df_cancelamentos.to_excel(xlsx_cancelation_path, engine='openpyxl', index=False)



AttributeError: 'NoneType' object has no attribute 'head'

In [46]:
xlsx_cancel_path = r"C:\Users\raphael.almeida\Documents\Processos\placas_movimentacoes\df_cancelados.xlsx"
df_cancelados = df_cancelados.to_excel(xlsx_cancel_path, engine='openpyxl', index=False)

first function: NEW / ACTIVE

In [59]:

def process(df, df_ontem):
    # Normalizar chassi em ambos os dataframes
    for d in (df, df_ontem):
        if 'chassi' in d.columns:
            d['chassi'] = d['chassi'].astype(str).str.strip().str.upper()

    # Conjuntos para comparação
    set_ontem = set(df_ontem['chassi'].dropna().unique()) if 'chassi' in df_ontem.columns else set()

    # Classificar status_beneficio em df
    if 'chassi' in df.columns:
        df['status_beneficio'] = np.where(df['chassi'].isin(set_ontem), 'ATIVO', 'NOVO')
    if 'migration_from' not in df.columns:
        df['migration_from'] = 'NULL'

    
    return df, df_ontem

process(df,df_ontem)

 

(         placa             chassi  id_placa  id_veiculo  id_carroceria  \
 0      MGC1B11  9BVN4DAA0YE672083  17322444    17322444              0   
 1      MGA0I88  9BST4X2A043550744  17322241    17322241              0   
 2      MFJ8H57  9BFVCE1N3ABB42605  17322663    17322663              0   
 3      MFE4B62  9BSR4X2A053564937  17322315    17322315              0   
 4      BEX9G24  9ADG0942MMM477102     27469           0          27469   
 ...        ...                ...       ...         ...            ...   
 32903  QAB1870  94BL0262KKR004389      3485           0           3485   
 32904  MHB9F69  955L14639AS299249      3652           0           3652   
 32905  AXV7H35  9EP081630E1001931      3198           0           3198   
 32906  AJW6D00  955L1463EES359870      3194           0           3194   
 32907  ESU7G77  9BSP4X200C3805546    339080      339080              0   
 
        matricula  conjunto                                            unidade  \
 0           517

last function: refining movement NEW boards

In [60]:
# Refinar NOVO usando df_conferencia (historico por chassi+beneficio)
def refining_new_boards(df,df_conf):
    try:
        # Normalização de chaves
        for d in (df_conf):
            if 'chassi' in d.columns:
                d['chassi'] = d['chassi'].astype(str).str.strip().str.upper()
            if 'beneficio' in d.columns:
                d['beneficio'] = d['beneficio'].astype(str).str.strip().str.upper()
            if 'empresa' in d.columns:
                d['empresa'] = d['empresa'].astype(str).str.strip()
        # Garantir datetime
        if 'data_ativacao_beneficio' in df_conf.columns:
            df_conf['data_ativacao_beneficio'] = pd.to_datetime(df_conf['data_ativacao_beneficio'], errors='coerce')

        # Aplicar apenas nas linhas marcadas como NOVO e que tenham chassi+beneficio
        mask_novo = df['status_beneficio'].eq('NOVO')
        if mask_novo.any() and {'chassi','beneficio'}.issubset(df.columns) and not df_conf.empty and {'chassi','beneficio','data_ativacao_beneficio','status_beneficio','empresa'}.issubset(df_conf.columns):
            df_novos = df.loc[mask_novo, ['chassi','beneficio','empresa']].copy()
            df_novos['beneficio'] = df_novos['beneficio'].astype(str).str.strip().str.upper()
            df_novos['empresa'] = df_novos['empresa'].astype(str).str.strip()

            # Contagem de registros por par
            hist_counts = (df_conf
                .groupby(['chassi','beneficio'], as_index=False)
                .size()
                .rename(columns={'size':'hist_count'}))

            # Penúltimo registro por par: ordenar desc e pegar a 2a linha (index 1)
            df_conf_sorted = df_conf.sort_values('data_ativacao_beneficio', ascending=False)
            penult = (df_conf_sorted
                .groupby(['chassi','beneficio'], as_index=False)
                .nth(1)
                .reset_index(drop=False))
            # Selecionar colunas relevantes
            penult = penult[['chassi','beneficio','status_beneficio','empresa']].rename(columns={
                'status_beneficio':'status_penultimo',
                'empresa':'empresa_penultima'
            })

            # Merge info de histórico e penúltimo nas linhas NOVO
            df_novos = df_novos.merge(hist_counts, on=['chassi','beneficio'], how='left')
            df_novos = df_novos.merge(penult, on=['chassi','beneficio'], how='left')

            # Regras:
            # - hist_count <= 1 -> permanece NOVO
            # - hist_count > 1:
            #   - status_penultimo in status_filter_list -> REATIVAÇÃO
            #   - else: se empresa_penultima != empresa_atual -> MIGRAÇÃO (migration_from = empresa_penultima)
            #           senão -> RENOVAÇÃO
            cond_hist_many = (df_novos['hist_count'].fillna(0) > 1)
            cond_penult_cancel = df_novos['status_penultimo'].isin(status_filter_list)
            cond_mudou_empresa = df_novos['empresa_penultima'].notna() & df_novos['empresa'].notna() & (df_novos['empresa_penultima'] != df_novos['empresa'])

            # Inicial
            df_novos['novo_status'] = 'NOVO'
            df_novos['novo_migration_from'] = 'NULL'

            # REATIVAÇÃO
            reativ_mask = cond_hist_many & cond_penult_cancel
            df_novos.loc[reativ_mask, 'novo_status'] = 'REATIVAÇÃO'
            df_novos.loc[reativ_mask, 'novo_migration_from'] = 'NULL'

            # MIGRAÇÃO
            migr_mask = cond_hist_many & ~cond_penult_cancel & cond_mudou_empresa
            df_novos.loc[migr_mask, 'novo_status'] = 'MIGRAÇÃO'
            df_novos.loc[migr_mask, 'novo_migration_from'] = df_novos.loc[migr_mask, 'empresa_penultima']

            # RENOVAÇÃO
            renov_mask = cond_hist_many & ~cond_penult_cancel & ~cond_mudou_empresa
            df_novos.loc[renov_mask, 'novo_status'] = 'RENOVAÇÃO'
            df_novos.loc[renov_mask, 'novo_migration_from'] = 'NULL'

            # Aplicar de volta no df
            df.loc[mask_novo, 'status_beneficio'] = df_novos['novo_status'].values
            df.loc[mask_novo, 'migration_from'] = df_novos['novo_migration_from'].values
    except Exception as e:
        logging.info(f"Falha ao refinar NOVO com histórico: {e}")

    return df

refining_new_boards(df,df_conferencia)

    


,placa,chassi,id_placa,id_veiculo,id_carroceria,matricula,conjunto,unidade,consultor,status_beneficio,cliente,data_registro,data_ativacao_beneficio,suporte,data_filtro,empresa,migration_from
0,MGC1B11,9BVN4DAA0YE672083,17322444,17322444,0,5175,16639,OPENTRUCK CORRETORA,Gilvana Daros,ATIVO,APROSSIL - ASSOCIACAO DE PROPRIETARIOS DE CAMI...,2025-05-20,2025-06-02,Lucas Belmiro Mendes Santos,2025-08-27,Viavante,NaN
1,MGA0I88,9BST4X2A043550744,17322241,17322241,0,5175,16639,OPENTRUCK CORRETORA,Gilvana Daros,ATIVO,APROSSIL - ASSOCIACAO DE PROPRIETARIOS DE CAMI...,2025-05-20,2025-06-02,Lucas Belmiro Mendes Santos,2025-08-27,Viavante,NaN
2,MFJ8H57,9BFVCE1N3ABB42605,17322663,17322663,0,5175,16639,OPENTRUCK CORRETORA,Gilvana Daros,ATIVO,APROSSIL - ASSOCIACAO DE PROPRIETARIOS DE CAMI...,2025-05-20,2025-06-02,Lucas Belmiro Mendes Santos,2025-08-27,Viavante,NaN
3,MFE4B62,9BSR4X2A053564937,17322315,17322315,0,5175,16639,OPENTRUCK CORRETORA,Gilvana Daros,ATIVO,APROSSIL - ASSOCIACAO DE PROPRIETARIOS DE CAMI...,2025-05-20,2025-06-02,Lucas Belmiro Mendes Santos,2025-08-27,Viavante,NaN
4,BEX9G24,9ADG0942MMM477102,27469,0,27469,1023,20469,UNIDADE LONDRINA,Daiane Cristina Veiga da Silva,ATIVO,TRANSPORTADORA SEMCHECHEM LTDA,2025-07-14,2025-07-14,Lucas Belmiro Mendes Santos,2025-08-27,Viavante,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32903,QAB1870,94BL0262KKR004389,3485,0,3485,975,1074,UNIDADE CAMPO GRANDE,Icaro Gomes De Oliveira,ATIVO,CEREALISTA ANZIBAS LTDA,2025-08-18,2025-08-19,Diego Alejandro Cortes Moya,2025-08-27,Tag,NaN
32904,MHB9F69,955L14639AS299249,3652,0,3652,993,1116,MICRO A - R A SERVICOS & CONSULTORIA LTDA,Rodrigo Almeida,ATIVO,RAPIDO EXPRESSO EVOLOG TRANSPORTES LTDA,2025-08-18,2025-08-18,Lucas Belmiro Mendes Santos,2025-08-27,Tag,NaN
32905,AXV7H35,9EP081630E1001931,3198,0,3198,1007,1134,MICRO B - T.V PASQUINI SERVICOS ADMINISTRATIVO...,MF - TIAGO PASQUINI,ATIVO,PADROEIRA BR. LOCAÇÕES LTDA,2025-08-18,2025-08-19,Lucas Belmiro Mendes Santos,2025-08-27,Tag,NaN
32906,AJW6D00,955L1463EES359870,3194,0,3194,1007,1134,MICRO B - T.V PASQUINI SERVICOS ADMINISTRATIVO...,MF - TIAGO PASQUINI,ATIVO,PADROEIRA BR. LOCAÇÕES LTDA,2025-08-18,2025-08-19,Lucas Belmiro Mendes Santos,2025-08-27,Tag,NaN


In [61]:
df_novos = df[df['status_beneficio']=='NOVO']
df_ativos = df[df['status_beneficio']=='ATIVO']

print(df_novos.shape[0])
print(df_ativos.shape[0])


106
32802


loading

In [49]:
xlsx_path = r"C:\Users\raphael.almeida\Documents\Processos\placas_movimentacoes\df_tratado.xlsx"
df_ativacoes_tratado = df.to_excel(xlsx_path, engine='openpyxl', index=False)



bkp


In [ ]:
# IMPORTANDO MÓDULOS E PACOTES
import pandas as pd
import numpy as np
import datetime as dt
import logging
import numpy as np
import os
import awswrangler as awr

xlsx_ontem = r"C:\Users\raphael.almeida\Documents\Processos\placas_movimentacoes\placas_movimentacoes_ontem.xlsx"
xlsx = r"C:\Users\raphael.almeida\Documents\Processos\placas_movimentacoes\placas_movimentacoes.xlsx"

df_ontem = pd.read_excel(xlsx_ontem, engine='openpyxl', sheet_name= 'ATIVAÇÕES')
df = pd.read_excel(xlsx, engine='openpyxl')
status_filter_list = ['CANCELADO', 'CANCELADA', 'FINALIZADO', 'FINALIZADA', 'NAO RENOVADO']

path = r"C:\Users\raphael.almeida\Documents\Processos\placas_movimentacoes"
dir_query = os.path.join(path,'sql', 'listagem_mestra.sql')

with open(dir_query, 'r') as file:
    query = file.read()

df_conferencia = awr.athena.read_sql_query(query, database='silver')

class ChassiProcessor:
    
    def process(df, df_ontem, df_conf):
        # Normalizar chassi em ambos os dataframes
        for d in (df, df_ontem):
            if 'chassi' in d.columns:
                d['chassi'] = d['chassi'].astype(str).str.strip().str.upper()

        # Conjuntos para comparação
        set_hoje = set(df['chassi'].dropna().unique()) if 'chassi' in df.columns else set()
        set_ontem = set(df_ontem['chassi'].dropna().unique()) if 'chassi' in df_ontem.columns else set()

        # Classificar status_beneficio em df
        if 'chassi' in df.columns:
            df['status_beneficio'] = np.where(df['chassi'].isin(set_ontem), 'ATIVO', 'NOVO')
        if 'migration_from' not in df.columns:
            df['migration_from'] = 'NULL'

        # Construir df_cancelados: existiam ontem e não existem hoje
        if 'chassi' in df_ontem.columns:
            cancelados_mask = ~df_ontem['chassi'].isin(set_hoje)
            df_cancelados = df_ontem.loc[cancelados_mask].copy()
        else:
            df_cancelados = pd.DataFrame(columns=df_ontem.columns)

        # Refinar NOVO usando df_conferencia (historico por chassi+beneficio)
        try:

            # Normalização de chaves
            for d in (df_conf):
                if 'chassi' in d.columns:
                    d['chassi'] = d['chassi'].astype(str).str.strip().str.upper()
                if 'beneficio' in d.columns:
                    d['beneficio'] = d['beneficio'].astype(str).str.strip().str.upper()
                if 'empresa' in d.columns:
                    d['empresa'] = d['empresa'].astype(str).str.strip()
            # Garantir datetime
            if 'data_ativacao_beneficio' in df_conf.columns:
                df_conf['data_ativacao_beneficio'] = pd.to_datetime(df_conf['data_ativacao_beneficio'], errors='coerce')

            # Aplicar apenas nas linhas marcadas como NOVO e que tenham chassi+beneficio
            mask_novo = df['status_beneficio'].eq('NOVO')
            if mask_novo.any() and {'chassi','beneficio'}.issubset(df.columns) and not df_conf.empty and {'chassi','beneficio','data_ativacao_beneficio','status_beneficio','empresa'}.issubset(df_conf.columns):
                df_novos = df.loc[mask_novo, ['chassi','beneficio','empresa']].copy()
                df_novos['beneficio'] = df_novos['beneficio'].astype(str).str.strip().str.upper()
                df_novos['empresa'] = df_novos['empresa'].astype(str).str.strip()

                # Contagem de registros por par
                hist_counts = (df_conf
                    .groupby(['chassi','beneficio'], as_index=False)
                    .size()
                    .rename(columns={'size':'hist_count'}))

                # Penúltimo registro por par: ordenar desc e pegar a 2a linha (index 1)
                df_conf_sorted = df_conf.sort_values('data_ativacao_beneficio', ascending=False)
                penult = (df_conf_sorted
                    .groupby(['chassi','beneficio'], as_index=False)
                    .nth(1)
                    .reset_index(drop=False))
                # Selecionar colunas relevantes
                penult = penult[['chassi','beneficio','status_beneficio','empresa']].rename(columns={
                    'status_beneficio':'status_penultimo',
                    'empresa':'empresa_penultima'
                })

                # Merge info de histórico e penúltimo nas linhas NOVO
                df_novos = df_novos.merge(hist_counts, on=['chassi','beneficio'], how='left')
                df_novos = df_novos.merge(penult, on=['chassi','beneficio'], how='left')

                # Regras:
                # - hist_count <= 1 -> permanece NOVO
                # - hist_count > 1:
                #   - status_penultimo in status_filter_list -> REATIVAÇÃO
                #   - else: se empresa_penultima != empresa_atual -> MIGRAÇÃO (migration_from = empresa_penultima)
                #           senão -> RENOVAÇÃO
                cond_hist_many = (df_novos['hist_count'].fillna(0) > 1)
                cond_penult_cancel = df_novos['status_penultimo'].isin(status_filter_list)
                cond_mudou_empresa = df_novos['empresa_penultima'].notna() & df_novos['empresa'].notna() & (df_novos['empresa_penultima'] != df_novos['empresa'])

                # Inicial
                df_novos['novo_status'] = 'NOVO'
                df_novos['novo_migration_from'] = 'NULL'

                # REATIVAÇÃO
                reativ_mask = cond_hist_many & cond_penult_cancel
                df_novos.loc[reativ_mask, 'novo_status'] = 'REATIVAÇÃO'
                df_novos.loc[reativ_mask, 'novo_migration_from'] = 'NULL'

                # MIGRAÇÃO
                migr_mask = cond_hist_many & ~cond_penult_cancel & cond_mudou_empresa
                df_novos.loc[migr_mask, 'novo_status'] = 'MIGRAÇÃO'
                df_novos.loc[migr_mask, 'novo_migration_from'] = df_novos.loc[migr_mask, 'empresa_penultima']

                # RENOVAÇÃO
                renov_mask = cond_hist_many & ~cond_penult_cancel & ~cond_mudou_empresa
                df_novos.loc[renov_mask, 'novo_status'] = 'RENOVAÇÃO'
                df_novos.loc[renov_mask, 'novo_migration_from'] = 'NULL'

                # Aplicar de volta no df
                df.loc[mask_novo, 'status_beneficio'] = df_novos['novo_status'].values
                df.loc[mask_novo, 'migration_from'] = df_novos['novo_migration_from'].values
        except Exception as e:
            logging.info(f"Falha ao refinar NOVO com histórico: {e}")

        return df, df_ontem, df_cancelados


ChassiProcessor.process(df, df_ontem, df_conferencia)

xlsx_path = r"C:\Users\raphael.almeida\Documents\Processos\placas_movimentacoes\df_tratado.xlsx"
df_ativacoes_tratado = df.to_excel(xlsx_path, engine='openpyxl', index=False)




In [8]:
df_final_ativacoes_ativos = df_final_ativacoes[df_final_ativacoes['status_beneficio']=='ATIVO']
df_final_ativacoes_ativos.shape[0]

32908

In [9]:
df_final_ativacoes_ativos.head()

,placa,chassi,id_placa,id_veiculo,id_carroceria,matricula,conjunto,unidade,consultor,status_beneficio,cliente,data_registro,data_ativacao_beneficio,suporte,data_filtro,empresa,migration_from
31977,TAX8B12,9539J8TH9SR202365,15566883,15566883,0,5485,8355,UNIDADE LONDRINA,Gismary Orasmo,ATIVO,SANTORINI TRANSPORTE E LOGISTICA LTDA,2025-01-16,2025-01-16,Camilla Minho,2025-08-27,Viavante,NULL
31979,TAX8B23,9539J8TH0SR203680,15566726,15566726,0,5485,8355,UNIDADE LONDRINA,Gismary Orasmo,ATIVO,SANTORINI TRANSPORTE E LOGISTICA LTDA,2025-01-16,2025-01-16,Camilla Minho,2025-08-27,Viavante,NULL
31980,PHT5C27,97TD0N412K2001627,14227,0,14227,5488,8360,UNIDADE CUIABA,Isabella Santos,ATIVO,JAFRE RANGEL DE SOUZA,2025-01-16,2025-01-16,João Paulo Ribeiro Pinto,2025-08-27,Viavante,NULL
31982,OKH0A38,9BVAG30C8EE824746,17376787,17376787,0,5489,18138,UNIDADE VILHENA,Vilma Girioli,ATIVO,CLEUVERSON PAZ REIS,2025-06-10,2025-06-10,Camilla Minho,2025-08-27,Viavante,NULL
31984,RQP8I07,953998TH7NR202118,15928702,15928702,0,5493,8364,UNIDADE VILHENA,Vilma Girioli,ATIVO,JHONATAN MIRANDA PESSOA,2025-01-16,2025-01-16,VICTOR GABRIEL GOMES PICÃO,2025-08-27,Viavante,NULL


transformando as planilhas excel em dataframe

In [12]:
xlsx_ontem = r"C:\Users\raphael.almeida\Documents\Processos\placas_movimentacoes\placas_movimentacoes_ontem.xlsx"
xlsx = r"C:\Users\raphael.almeida\Documents\Processos\placas_movimentacoes\placas_movimentacoes.xlsx"

df_ontem = pd.read_excel(xlsx_ontem, engine='openpyxl', sheet_name= 'ATIVAÇÕES')
df = pd.read_excel(xlsx, engine='openpyxl')
status_filter_list = ['CANCELADO', 'CANCELADA', 'FINALIZADO', 'FINALIZADA', 'NAO RENOVADO']

In [20]:
df_ativacoes_tratado2 = Transform.board_status_treatment2(df, df_ontem, df_conf, status_filter_list)

In [18]:
xlsx_path = r"C:\Users\raphael.almeida\Documents\Processos\placas_movimentacoes\df_tratado.xlsx"
df_ativacoes_tratado.to_excel(xlsx_path, engine='openpyxl', index=False)

In [21]:
df_ativacoes_tratado2['status_beneficio'].unique()

array(['ATIVO'], dtype=object)

# LOAD

In [ ]:
# IMPORTANDO MÓDULOS E PACOTES
import logging
import openpyxl
import os
import pandas as pd
import logging


file_path = r"C:\Users\raphael.almeida\Documents\Processos\placas_acompanhamento\template\placas_movimentacoes.xlsx"
destination_dir = r"C:\Users\raphael.almeida\OneDrive - Grupo Unus\analise de dados - Arquivos em excel\Relatório de Ativações Placas"
destination_path = os.path.join(destination_dir, os.path.basename(file_path))

with pd.ExcelWriter(destination_path, engine='openpyxl') as writer:
    df_final_ativacoes.to_excel(writer, index=False, sheet_name='ATIVAÇÕES')


logging.info('\n ----------------------------------------------------------------------------------')
logging.info('\n Processo de Carregamento de Dados concluido com sucesso!')




    


2025-08-26 09:55:58,998 - INFO - 
 ----------------------------------------------------------------------------------
2025-08-26 09:55:59,000 - INFO - 
 Processo de Carregamento de Dados concluido com sucesso!
